In [49]:
# импорт необходимых библиотек
import pandas as pd

import requests

In [2]:
start_date = input('Введите дату начала в формате дд.мм.гггг')
finish_date = input('Введите дату конца в формате дд.мм.гггг')  # просим пользователя ввести даты начала и конца периода

Введите дату начала в формате дд.мм.гггг10.10.2022
Введите дату конца в формате дд.мм.гггг18.10.2022


In [3]:
start_date = pd.to_datetime(start_date, format='%d.%m.%Y')
finish_date = pd.to_datetime(finish_date, format='%d.%m.%Y')  # приводим введенные даты в формат даты

In [4]:
start_date = start_date.strftime('%m.%d.%Y')
finish_date = finish_date.strftime('%m.%d.%Y')  # приводим даты к формату, который требует сайт

In [50]:
# отправляем запрос на сайт
file = requests.get(f'https://cbbh.ba/CurrencyExchange/GetJsonForPeriod?dateFrom={start_date}&dateTo={finish_date}')

In [51]:
data = file.json()  #сохраняем полученные данные в json

In [52]:
df = pd.json_normalize(data, "CurrencyExchangeItems", ["Date"])

In [53]:
df = df[df["AlphaCode"] == "USD"][["Date", "Middle"]]

In [54]:
df = df.rename(columns={"Date": "Дата", "Middle": "Курс"})

In [55]:
# приводим все к правильному типу данных
df['Дата'] = pd.to_datetime(df['Дата'])
df['Курс'] = df['Курс'].str.replace(',', '.')
df['Курс'] = df['Курс'].astype(float)

In [56]:
# приводим дату к формату дд/мм/гггг
df['Дата'] = df['Дата'].dt.strftime('%d/%m/%Y')

In [57]:
df

,Дата,Курс
13,11/10/2022,2.016943
31,12/10/2022,2.011550
49,13/10/2022,2.015073
67,14/10/2022,2.008245
85,15/10/2022,2.012792
103,18/10/2022,2.008245
